# Sparameters

gdsfactory provides you with a Lumerical FDTD interface to calculate Sparameters

by default another repo [gdslib](https://gdslib.readthedocs.io/en/latest/index.html) stores the Sparameters 

You can chain the Sparameters to calculate solve of larger
circuits using a circuit solver such as:

- Lumerical interconnect
- [simphony (open source)](https://simphonyphotonics.readthedocs.io/en/latest/)


If the Sparameters exists in `gdslib` you can access them really fast.

In [ ]:
# NBVAL_SKIP
import pp
import pp.sp as sp
import lumapi

s = lumapi.FDTD()

In [ ]:
pp.c.LIBRARY.factory.keys()

In [ ]:
components =  ['bend_euler', 'bend_s', 'coupler', 'coupler_ring', 'crossing', 'mmi1x2', 'mmi2x2', 'taper',  'straight']
need_review = []

for component_name in components:
    c = pp.c.LIBRARY.get_component(component_name)
    sp.write(c, run=False, session=s)
    response = input(f'does the simulation for {component_name} look good? (y/n)')
    if response.upper()[0] == 'N':
        need_review.append(c)

In [ ]:
components =  ['crossing', 'mmi1x2', 'mmi2x2', 'taper',  'straight']
need_review = []

for component_name in components:
    c = pp.c.LIBRARY.get_component(component_name)
    sp.write(c, run=False, session=s)
    response = input('does the simulation look good? (y/n)')
    if response.upper()[0] == 'N':
        need_review.append(c)

In [ ]:
sp.plot(pp.components.mmi1x2(), keys=['S23m', 'S13m'], logscale=True)

In [ ]:
sp.write(pp.components.mmi1x2(), layer_to_thickness_nm={(1,0): 210})

In [ ]:
layer_to_thickness_nm = pp.tech.LAYER_STACK.get_layer_to_thickness_nm()
component = pp.c.bend_circular(radius=3)
component = component.copy()
component.remove_layers(component.layers - set(layer_to_thickness_nm.keys()))
component._bb_valid = False

c = pp.extend.extend_ports(component=component, length=1.)
c.show()
gdspath = c.write_gds()

c

gdsfactory can also compute the Sparameters of a component that have not been simulated before.

In [ ]:
# NBVAL_SKIP

cs = [pp.components.coupler_ring(gap=gap, radius=radius) for gap in [0.15, 0.2, 0.3] for radius in [5, 10]]

for c in cs:
    c.show()
    print(c)
    sp.write(c)

To debug a simulation you can create a Lumerical session outside the simulator, pass it to the simulator, and use `run=False` flag

In [ ]:
# NBVAL_SKIP
import lumapi
s = lumapi.FDTD()
c = pp.components.straight()
sp.write(c, run=False, session=s)

By default gdsfactory uses the generic LayerStack for 220nm height silicon layer.

You can also define your a different LayerStack

In [ ]:
import pp

class LayerStackExample(pp.tech.LayerStack):
    WG = pp.tech.LayerLevel(layer=(2,0), thickness_nm=400.0, zmin_nm=0.0, material='sin')
    
ls = LayerStackExample()

In [ ]:
ls_tmin = LayerStackExample()
ls_tmin.WG.thickness_nm -= 5
print(ls_tmin.WG.thickness_nm)

In [ ]:
ls_tmax = LayerStackExample()
ls_tmax.WG.thickness_nm += 5
print(ls_tmin.WG.thickness_nm)

In [ ]:
ls.get_layer_to_material()

In [ ]:
c = pp.components.straight(layer=(2,0))
sp = sp.write(c, layer_stack=ls, session=s)
c.plot()
pp.sp.plot(c)